# IBM & DataStax Demo – Banking AI Agent

## Prerequisites

- Install dependencies (see `README.md`)
- DataStax Astra account
- Watsonx.AI or OpenAI API key

## Part I – Astra Setup

- Sign up for DataStax Astra  
- Create a database and vector-enabled collection (NVIDIA model)  
- Upload a document to the collection  

## Part II – Langflow RAG

- Build a Langflow RAG flow and connect it to Astra  
- Enable NVIDIA reranker  
- Trigger the flow via Langflow API  

## Part III – Agents & NoSQL

- Create a CQL table for banking transactions  
- Insert sample data  
- Build a Banking Agent Flow in Langflow  
- Integrate Astra DB Tools into the agent  
- Run the flow via Langflow API  


# Why Astra DB

### Low Latency / High Throughput

Applications that need to write thousands of records per second or read data within milliseconds — even across multiple terabytes — benefit from Cassandra’s data model, which is intentionally optimized for high-performance reads and writes.

### High Availability

Applications that cannot afford downtime require data replication across multiple data centers. Every Astra DB instance automatically runs across three availability zones within a cloud region, achieving 99.99% uptime. Optionally, databases can span two (or more) regions, further increasing availability to 99.999%. This replication happens automatically without requiring changes to the application.

### Massive Storage with Linear Scalability

Astra DB can store terabytes of data while maintaining consistent read and write performance. Thanks to intelligent data partitioning, records are organized in a way that allows Cassandra to scale linearly and efficiently, even as data volume grows.

## Astra DB

Beyond the Apache Cassandra features, by running applications on DataStax Astra, the companies will also have additional benefits:

### Reduced Operational Overhead

Astra DB is a fully managed service that alleviates the need for teams to perform operational tasks such as server management, scaling, or maintenance. This allows developers to focus on building features rather than managing the database infrastructure, enhancing productivity.

### Multi-Cloud Flexibility

Astra DB supports deployment across multiple cloud environments, providing businesses with the flexibility to operate in a hybrid or multi-cloud strategy. This flexibility enhances disaster recovery and ensures better performance based on geographic demands. Currently, users can create databases on AWS, MIcrosoft Azure and GCP in pultiple regions.

### Enhanced Security and Compliance

With built-in security features and compliance with industry standards (including HIPAA for healthcare and PCI for banking), Astra DB provides a secure environment for managing sensitive data, which is critical for regulated industries.

# Astra DB — Key Characteristics and Application Fit

| Astra DB Characteristic | Application Needs                       | Why It Matters                                                      | Example Use Cases                                                                                                  |
| ----------------------- | --------------------------------------- | ------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------ |
| **Scaling**             | High Throughput, High Volume, Real-Time | Supports heavy writes and reads at scale                            | AI memory, chatbot sessions, event streaming, IoT, log analytics, time series, transaction systems, feature stores |
| **Availability**        | Mission-Critical Operations             | No data loss, always-on availability                                | E-commerce, caching, inventory management, healthcare systems                                                      |
| **Distribution**        | Global Presence, Workload Mobility      | Compliance (e.g., GDPR), enhanced customer experience               | Banking, media streaming, logistics, retail                                                                        |
| **Cloud Native**        | Managed Services                        | API-first architecture, multi-model support, multi-cloud deployment | Mobile apps, cloud modernization projects                                                                          |
| **AI Ready**            | Knowledge Layer Integration             | Vector search, re-ranking, LLM integration                          | Retrieval-Augmented Generation (RAG), AI agents, customer service automation                                       |


# Starting the demo

What we are going to build:


<img src="img/lf_demo.png" alt="Demo" width="600"/>


## Installing dependencies

Check the [README.md](./README.md) to set up your enviroment before startinge the execution of this notebook.

## Creating Your DataStax Astra Account

> **INFO:** Every user on DataStax Astra receives $25 in credits *EVERY MONTH* to use Astra DB, Astra Streaming, and Langflow.

To get started, visit [astra.datastax.com](https://astra.datastax.com) and click **Sign Up** to begin registration.

<img src="img/signup.png" alt="Sign Up Page" width="600"/>

Once you've completed the sign-up process, you'll be able to log in to the Astra Dashboard.

# Part I - Astra DB Setup

## Creating a Database

After logging in on Astra Dashboard, click on the "Create Database" button.

<img src="img/dashboard.png" alt="Start Database creation" width="600"/>

On the Create Database screen, fill

- Deployment Type: Serverless (Vector)
- Provider: AWS
- Region: us-east-2

<img src="img/create_db.png" alt="DB Creation" width="600"/>

> INFO: It is possible to create DBs on AWS, Azure and GCP. To check all available regions, check our [documentation](https://docs.datastax.com/en/astra-db-serverless/databases/regions.html)

After 2 or 3 minutes, your database will be ready to start using it.

<img src="img/db_created.png" alt="DB Created" width="600"/>

More info available [here](https://docs.datastax.com/en/astra-db-serverless/databases/create-database.html)


## Creating the Collection

> INFO: Astra DB can store __collections__ and __tables__
>
> [__Collections__](https://docs.datastax.com/en/astra-db-serverless/api-reference/collections.html) are used to store JSON documents with a semi structured data model. It is compatible with Mongo DB and [MongooseJS](https://docs.datastax.com/en/astra-db-serverless/integrations/data-api-with-mongoosejs.html). 
>
> [__Tables__](https://docs.datastax.com/en/astra-db-serverless/cql/develop-with-cql.html) are used to store transactional data that requires high throughput and low latencies. Due to its data model it is possible to achieve tens of thousands operations per second. 
>
> Both models are compatible with __Vector Search__


On the DB Dashboard, click on __"Data Explorer"__ and then click on __"Create Collection +"__

<img src="img/start_create_collection.png" alt="Start collection creation" width="600"/>

The screen for the collection creation should be filled with:

- Collection name: banking_collection
- Vector-enabled collection: Activated
- Embedding generation method: NVIDIA (The embedding model and dimension will be filled automatically)
- Similarity Metric: Cosine (Dot Product and Euclidean are additional options)

<img src="img/create_collection.png" alt="Create collection" width="600"/>


> ### Vectorize
> 
> 
> When creating a collection, you can automate the embedding generation process using [__Vectorize__](https://docs.datastax.com/en/astra-db-serverless/databases/embedding-generation.html).
>
> In this example, we're using NVIDIA models provided by DataStax. However, you can also integrate with other model providers such as OpenAI, Azure OpenAI, Hugging Face, and more.
> 
> With this setup, all embedding generation—whether during data loading or querying—is seamlessly handled at the database layer.


## Loading Data

Astra supports multiple data loading methods. For this demo, we’ll use [**LangChain**](https://python.langchain.com/docs/integrations/vectorstores/astradb/) to highlight the developer-friendly experience for those who want full control of the pipeline.

## Get the Token and API Endpoint

1. **Set up your environment file**  
   Copy the `_env_sample` file and rename it to `.env`. This is where you’ll store your Astra credentials.

2. **Locate your database details**  
   In the Astra Dashboard, go to the `astra_ibm_demo` database, click the **Overview** tab, and find **Database Details** on the right panel.  
   <img src="img/token.png" alt="Token" width="600"/>

3. **Copy and store the API Endpoint**  
   Copy the **API Endpoint** and paste it into your `.env` file.

4. **Generate an application token**  
   In the **Application Tokens** section, click **Generate Token**. Give it a name and create the token.  
   <img src="./img/generate_token.png" alt="Token" width="600"/>

5. **Copy the token and update your `.env`**  
   Click **Copy**, then paste the token into your `.env` file. It should look like this:

   ```bash
   ASTRA_DB_APPLICATION_TOKEN="<your token>"
   ASTRA_DB_API_ENDPOINT="<your db endpoint>"
   IBM_WATSON_TOKEN=""
   OPENAI_API_TOKEN=
   ```

6. **Verify your environment setup**  
   Ensure all variables are correctly saved in `.env`.

---

Need this version formatted for a slide or internal doc?



In [1]:
#Make sure the environment variables are set
import os
from dotenv import load_dotenv
load_dotenv(override=True)

if os.getenv("ASTRA_DB_APPLICATION_TOKEN") is None:
    raise ValueError("Environment variable ASTRA_DB_APPLICATION_TOKEN not set")

if os.getenv("ASTRA_DB_APPLICATION_TOKEN")[:8] != "AstraCS:":
    raise ValueError("Environment variable ASTRA_DB_APPLICATION_TOKEN invalid format")

if ".apps.astra.datastax.com" not in os.getenv("ASTRA_DB_API_ENDPOINT"):
    raise ValueError("Environment variable ASTRA_DB_API_ENDPOINT invalid")

print(f'Astra Token: {os.getenv("ASTRA_DB_APPLICATION_TOKEN")[:10]}...{os.getenv("ASTRA_DB_APPLICATION_TOKEN")[-5:]}')
print(f'Astra Endpoint: {os.getenv("ASTRA_DB_API_ENDPOINT")}')
print("Good to go!")

Astra Token: AstraCS:bf...35ec6
Astra Endpoint: https://0df6bdb1-a6ca-4420-ba2e-f28550b3d178-us-east-2.apps.astra.datastax.com
Good to go!


### Data Load

We’ll use public documents from Chase Bank as the source for question answering and Agentic RAG.

The files are located in the `docs` folder.

In [2]:
# Importing necessary libraries
from langchain_astradb import AstraDBVectorStore
from langchain_astradb.utils.astradb import HybridSearchMode
import pandas as pd


In [3]:
# Initializing the AstraDBVectorStore
vector_store = AstraDBVectorStore(
    collection_name="banking_collection",
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    autodetect_collection=True,
    hybrid_search=HybridSearchMode.OFF
)

/Users/samuel.matioli/work/astra-ibm-demo-2/venv/lib/python3.11/site-packages/langchain_astradb/vectorstores.py:957: UserWarning: Collection supports lexical; however, autodetect encountered documents without a $lexical. The $lexical field will be set for new documents, but the pre-existing documents might not be reached by lexical search when running hybrid search mode.
  self.document_codec = _detect_document_codec(


In [17]:
# Cleaning up the collection to start fresh
# WARNING: This will delete all documents in the collection
vector_store.clear() 

In [4]:
# Loading the documents from the specified directory
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the documents from the specified directory
loader = DirectoryLoader(
    "./docs",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=80,
)
documents = loader.load()
texts = text_splitter.split_documents(documents)
print(f"Loaded {len(texts)} chunks from the directory.")

# Add the documents to the vector store
vector_store.add_documents(texts)
print(f"Added {len(texts)} chunks to the vector store.")

Loaded 852 chunks from the directory.
Added 852 chunks to the vector store.


### Querying Data

With the knowledge base loaded, we can now query the vector store to retrieve passages relevant to a given question—this is the core of [Retrieval-Augmented Generation (RAG)](https://www.datastax.com/blog/what-is-rag-retrieval-augmented-generation).

Astra DB supports [millions of embeddings](https://www.datastax.com/press-release/wikimedia-deutschland-launches-ai-knowledge-project-in-collaboration-with-datastax-built-with-nvidia-ai), but for optimal performance and accuracy, filtering by metadata is highly recommended.

Next, we'll apply a metadata filter to target a specific document.

In our first query, we'll return the top 3 most relevant chunks using vector search only.

In [5]:
QUERY = "Can I refund a lost ticket?"
SOURCE_DOCUMENT = "docs/BGC10981_SapphireReserve_VisaInfinite.pdf"

def print_docs(docs):
    for doc in docs:
        print(f"Title: {doc.metadata['title']}")
        print(f"Subject: {doc.metadata['subject']}")
        print(f"Page Content: {doc.page_content}")
        print("-" * 80)

def astra_vector_search(query, source, k=3):
    # Perform a similarity search with the Hybrid search disabled
    vector_store = AstraDBVectorStore(
        collection_name="banking_collection",
        api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
        token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
        autodetect_collection=True,
        hybrid_search=HybridSearchMode.OFF # Hybrid search disabled, so basic vector search is used
    )
    docs = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "filter": {"source": source},
            "k": k
        }
    ).invoke(query)
    
    return docs

# Create a DataFrame to display the docs in a table format
docs = astra_vector_search(QUERY,SOURCE_DOCUMENT,3)
print_docs(docs)

{'$vectorize': 'Can I refund a lost ticket?'}
{'source': 'docs/BGC10981_SapphireReserve_VisaInfinite.pdf'}
Title: Guide to Benefits
Subject: Guide
Page Content: repatriate the remains. All costs are Your responsibility.
•  Emergency Ticket Replacement helps You through Your carrier’s 
lost ticket reimbursement process and assists in the delivery of a 
replacement ticket to You, should You lose Your ticket. All costs are 
Your responsibility.
•  Lost Luggage Locator Service helps You through the Common 
Carrier’s claim procedures or can arrange shipment of replacement 
items if an airline or Common Carrier loses Your checked luggage.
--------------------------------------------------------------------------------
Title: Guide to Benefits
Subject: Guide
Page Content: 21
If you're outside of the US, call collect at 1-804-281-5772
Lost Luggage Reimbursement
The Lost Luggage benefit provides reimbursement for the difference 
between the actual cash value (replacement cost less depreciation)

The initial results are relevant but don’t directly answer the question.

To improve accuracy and response quality, let’s enable **reranking**.

### Integrated Reranking

In RAG applications, initial results from Astra may need refinement for better accuracy.

Astra’s API [supports built-in reranking](https://docs.datastax.com/en/astra-db-serverless/api-reference/document-methods/find-and-rerank.html), delivering more relevant documents by default—no extra setup required.

> **INFO:** Astra DB Hybrid Search uses server-side reranking powered by NVIDIA NeMo Retriever microservices—part of NVIDIA AI Enterprise via NVIDIA NIM.  
> Learn more in [this blog post](https://www.datastax.com/blog/introducing-astra-db-hybrid-search).

   <img src="./img/reranker.png" alt="Rerankers" width="400"/>

Now, let’s rerun the same query and compare the results.

In [6]:
def astra_vector_search_and_rerank(query, source, k=3):
    # Lets recreate the vector store with hybrid search enabled
    vector_store = AstraDBVectorStore(
        collection_name="banking_collection",
        api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
        token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
        autodetect_collection=True,
        hybrid_search=HybridSearchMode.ON # Hybrid search enabled
    )

    docs_reranked = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "filter": {"source": source},
            "k": k
        }
    ).invoke(query)
    return docs_reranked

docs_reranked = astra_vector_search_and_rerank(QUERY,SOURCE_DOCUMENT,3)
print_docs(docs_reranked)

/Users/samuel.matioli/work/astra-ibm-demo-2/venv/lib/python3.11/site-packages/langchain_astradb/vectorstores.py:2679: BetaFeatureWarning: Method 'Collection.find_and_rerank' is in beta and might undergo signature or behaviour changes in the future.
  hybrid_reranked_results = self.astra_env.collection.find_and_rerank(


Title: Guide to Benefits
Subject: Guide
Page Content: repatriate the remains. All costs are Your responsibility.
•  Emergency Ticket Replacement helps You through Your carrier’s 
lost ticket reimbursement process and assists in the delivery of a 
replacement ticket to You, should You lose Your ticket. All costs are 
Your responsibility.
•  Lost Luggage Locator Service helps You through the Common 
Carrier’s claim procedures or can arrange shipment of replacement 
items if an airline or Common Carrier loses Your checked luggage.
--------------------------------------------------------------------------------
Title: Guide to Benefits
Subject: Guide
Page Content: 41
If you're outside of the US, call collect at 1-804-281-5772
necessary medical treatment, to the airport, terminal or station of 
departure, and/or between the arrival airport, terminal or station 
and Your residence. This does not include transportation in vehicles 
operated by a medical facility or specifically designed to tr

Lets put them side by side:

In [7]:
QUERY = "Is lugagge lost covered?"
SOURCE_DOCUMENT = "docs/BGC10981_SapphireReserve_VisaInfinite.pdf"

docs = astra_vector_search(QUERY,SOURCE_DOCUMENT,3)
docs_reranked = astra_vector_search_and_rerank(QUERY,SOURCE_DOCUMENT,3)

# Prepare data for the table
data = {
    "Docs": [doc.page_content for doc in docs],
    "Docs Reranked": [doc.page_content for doc in docs_reranked]
}

# Create a DataFrame
comparison_df = pd.DataFrame(data)

with pd.option_context('display.max_colwidth', None):
    display(comparison_df)
    
    

{'$vectorize': 'Is lugagge lost covered?'}
{'source': 'docs/BGC10981_SapphireReserve_VisaInfinite.pdf'}


/Users/samuel.matioli/work/astra-ibm-demo-2/venv/lib/python3.11/site-packages/langchain_astradb/vectorstores.py:2679: BetaFeatureWarning: Method 'Collection.find_and_rerank' is in beta and might undergo signature or behaviour changes in the future.
  hybrid_reranked_results = self.astra_env.collection.find_and_rerank(


,Docs,Docs Reranked
0,"three (3) years after the time for giving proof of loss. Further, no \nlegal action may be brought against the Provider unless all the terms \nof the Guide to Benefits have been complied with fully. \n• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Federal Insurance Company. Coverage \nmay not be available in all states or certain terms may be different \nwhere required by state law.",sequence to the loss. War does not include terrorism.\nWhat’s Not Covered?\nThe Lost Luggage benefit does not apply to loss or theft of \nthe following: \n• Items that have been removed from Carry–On Baggage by You and \ninadvertently left behind on the Common Carrier or Cruise Line\n• Documents or valuable papers\n• Money\n• Securities\n• Tickets\n• Checks\n• Travelers’ checks\n• Furs\nHow Do You File a Claim?\nFollow these steps to file Your claim:
1,• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Indemnity Insurance Company of North \nAmerica (“Provider”). Coverage may not be available in all states or \ncertain terms may be different where required by state law. \n• No legal action for a claim may be brought against the Provider \nuntil sixty (60) days after the Provider receives Proof of Loss.,"her Account for a Trip, each Covered Person is eligible to receive \nthe benefit. \nPlease Pay Special Attention to These Conditions of \nthe Lost Luggage Benefit: \n• If Your Checked Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, the loss must be reported to the \nCommon Carrier or Cruise Line within the Common Carrier’s or \nCruise Line’s required timeframe for notification to be eligible for \nthis benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the"
2,• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Indemnity Insurance Company of \nNorth America (“Provider”). Coverage may not be available in all \nstates or certain terms may be different where required by state law. \n• No legal action for a claim may be brought against the Provider \nuntil sixty (60) days after the Provider receives Proof of Loss.,"this benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, it must be reported to the Common \nCarrier or Cruise Line as soon as You exit the Conveyance.\n• This benefit does not apply to any loss caused by or resulting from, \ndirectly or indirectly: War, undeclared war, civil war, insurrection, \nItems You inadvertently leave behind on the Common Carrier are not \nconsidered lost or stolen."


We can see that the reranked results are more relevant to the query.

### Adding Lexical Search

Semantic search excels at understanding meaning, but sometimes you need precision—like matching keywords, brand names, acronyms, or exact phrases. That’s where **lexical search** comes in.

Astra DB supports lexical filtering using the `$sort` clause, which can be combined with vector search for a **hybrid approach**—balancing semantic relevance with exact keyword control.

> **INFO:** Learn how hybrid search works in this [article](https://www.datastax.com/blog/improve-python-search-relevance-with-astra-db-hybrid-search).

In the next example, we'll apply lexical filters to return results containing specific terms.


In [8]:
def astra_vector_search_rerank_lexical(query, source, lexical_filter, k=3):
    # Lets recreate the vector store with hybrid search enabled
    vector_store = AstraDBVectorStore(
        collection_name="banking_collection",
        api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
        token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
        autodetect_collection=True,
        hybrid_search=HybridSearchMode.ON
    )

    docs_reranked_lexical = vector_store.similarity_search(
        query,
        k=k,
        filter={"source": source},
        lexical_query=lexical_filter
    )
    
    return docs_reranked_lexical

docs_lexical = astra_vector_search_rerank_lexical(QUERY,SOURCE_DOCUMENT,"police",3)
print_docs(docs_lexical)


/Users/samuel.matioli/work/astra-ibm-demo-2/venv/lib/python3.11/site-packages/langchain_astradb/vectorstores.py:2679: BetaFeatureWarning: Method 'Collection.find_and_rerank' is in beta and might undergo signature or behaviour changes in the future.
  hybrid_reranked_results = self.astra_env.collection.find_and_rerank(


Title: Guide to Benefits
Subject: Guide
Page Content: sequence to the loss. War does not include terrorism.
What’s Not Covered?
The Lost Luggage benefit does not apply to loss or theft of 
the following: 
•  Items that have been removed from Carry–On Baggage by You and  
inadvertently left behind on the Common Carrier or Cruise Line
• Documents or valuable papers
• Money
• Securities
• Tickets
• Checks
• Travelers’ checks
• Furs
How Do You File a Claim?
Follow these steps to file Your claim:
--------------------------------------------------------------------------------
Title: Guide to Benefits
Subject: Guide
Page Content: her Account for a Trip, each Covered Person is eligible to receive 
the benefit. 
Please Pay Special Attention to These Conditions of 
the Lost Luggage Benefit: 
•  If Your Checked Baggage is lost, stolen, or damaged by the 
Common Carrier or Cruise Line, the loss must be reported to the 
Common Carrier or Cruise Line within the Common Carrier’s or 
Cruise Line’s re

In [23]:
QUERY = "Is lugagge lost covered?"
SOURCE_DOCUMENT = "docs/BGC10981_SapphireReserve_VisaInfinite.pdf"
LEXICAL_FILTER = "police"

docs = astra_vector_search(QUERY,SOURCE_DOCUMENT,3)
docs_reranked = astra_vector_search_and_rerank(QUERY,SOURCE_DOCUMENT,3)
docs_lexical = astra_vector_search_rerank_lexical(QUERY,SOURCE_DOCUMENT,LEXICAL_FILTER,3)
# Prepare data for the table
data = {
    "Docs": [doc.page_content for doc in docs],
    "Docs Reranked": [doc.page_content for doc in docs_reranked],
    "Docs Lexical": [doc.page_content for doc in docs_lexical]
}

# Create a DataFrame
comparison_df = pd.DataFrame(data)

with pd.option_context('display.max_colwidth', None):
    display(comparison_df)
    
    

{'$vectorize': 'Is lugagge lost covered?'}
{'source': 'docs/BGC10981_SapphireReserve_VisaInfinite.pdf'}


/Users/samuel.matioli/work/astra-ibm-demo-2/venv/lib/python3.11/site-packages/langchain_astradb/vectorstores.py:2679: BetaFeatureWarning: Method 'Collection.find_and_rerank' is in beta and might undergo signature or behaviour changes in the future.
  hybrid_reranked_results = self.astra_env.collection.find_and_rerank(


,Docs,Docs Reranked,Docs Lexical
0,"three (3) years after the time for giving proof of loss. Further, no \nlegal action may be brought against the Provider unless all the terms \nof the Guide to Benefits have been complied with fully. \n• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Federal Insurance Company. Coverage \nmay not be available in all states or certain terms may be different \nwhere required by state law.",sequence to the loss. War does not include terrorism.\nWhat’s Not Covered?\nThe Lost Luggage benefit does not apply to loss or theft of \nthe following: \n• Items that have been removed from Carry–On Baggage by You and \ninadvertently left behind on the Common Carrier or Cruise Line\n• Documents or valuable papers\n• Money\n• Securities\n• Tickets\n• Checks\n• Travelers’ checks\n• Furs\nHow Do You File a Claim?\nFollow these steps to file Your claim:,sequence to the loss. War does not include terrorism.\nWhat’s Not Covered?\nThe Lost Luggage benefit does not apply to loss or theft of \nthe following: \n• Items that have been removed from Carry–On Baggage by You and \ninadvertently left behind on the Common Carrier or Cruise Line\n• Documents or valuable papers\n• Money\n• Securities\n• Tickets\n• Checks\n• Travelers’ checks\n• Furs\nHow Do You File a Claim?\nFollow these steps to file Your claim:
1,• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Indemnity Insurance Company of North \nAmerica (“Provider”). Coverage may not be available in all states or \ncertain terms may be different where required by state law. \n• No legal action for a claim may be brought against the Provider \nuntil sixty (60) days after the Provider receives Proof of Loss.,"her Account for a Trip, each Covered Person is eligible to receive \nthe benefit. \nPlease Pay Special Attention to These Conditions of \nthe Lost Luggage Benefit: \n• If Your Checked Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, the loss must be reported to the \nCommon Carrier or Cruise Line within the Common Carrier’s or \nCruise Line’s required timeframe for notification to be eligible for \nthis benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the","her Account for a Trip, each Covered Person is eligible to receive \nthe benefit. \nPlease Pay Special Attention to These Conditions of \nthe Lost Luggage Benefit: \n• If Your Checked Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, the loss must be reported to the \nCommon Carrier or Cruise Line within the Common Carrier’s or \nCruise Line’s required timeframe for notification to be eligible for \nthis benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the"
2,• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Indemnity Insurance Company of \nNorth America (“Provider”). Coverage may not be available in all \nstates or certain terms may be different where required by state law. \n• No legal action for a claim may be brought against the Provider \nuntil sixty (60) days after the Provider receives Proof of Loss.,"this benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, it must be reported to the Common \nCarrier or Cruise Line as soon as You exit the Conveyance.\n• This benefit does not apply to any loss caused by or resulting from, \ndirectly or indirectly: War, undeclared war, civil war, insurrection, \nItems You inadvertently leave behind on the Common Carrier are not \nconsidered lost or stolen.","this benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, it must be reported 

### Recap

In this exercise, you learned how to:

- Set up an Astra account and database  
- Create a collection with **Vectorize** for embedding generation  
- Retrieve your Astra access token and API endpoint  
- Load data using Python and LangChain  
- Use vector search to retrieve relevant content  
- Improve results with built-in **reranking**  
- Combine **vector** and **lexical** search for precision and relevance  